In [ ]:
# Import necessary libraries
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime
from flask import escape, jsonify

# Initialize Firebase Admin
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:


# Function definitions (fetch_loan_products, fetch_overdue_loans, etc.) remain the same
def fetch_loan_products():
    """
    Fetch all loan products and their penalty percentages.

    :return: A dictionary mapping product IDs to their penalty percentages
    """
    products_ref = db.collection('loanProduct')
    products = products_ref.stream()
    product_penalty_percentages = {}

    for product in products:
        product_data = product.to_dict()
        product_penalty_percentages[product.id] = product_data.get('PenaltyPercentageDay', 0)

    return product_penalty_percentages


def fetch_overdue_loans():
    """
    Fetch loans that are overdue.

    :return: A list of overdue loan documents
    """
    today = datetime.now().date()
    loans_ref = db.collection('farmerSimpleLoanApply')
    overdue_loans = []

    today = datetime.now().date()
    today_datetime = datetime.combine(today, datetime.min.time())

    # Use 'today_datetime' instead of 'today' for Firestore operations
    query = loans_ref.where('extendedDate', '<', today_datetime).stream()

    for loan in query:
        overdue_loans.append(loan)

    return overdue_loans

def calculate_daily_penalty(loan_details, penalty_percentage_per_day):
    """
    Calculate the daily penalty for the current day and add it to the accumulated penalty.
    :param loan_details: A dictionary containing loan details, including interest amount and accumulated penalty.
    :param penalty_percentage_per_day: Penalty percentage per day
    :return: The updated total penalty amount
    """
    # Extract necessary details from loan_details
    due_date = loan_details['DueDate'].date() if 'DueDate' in loan_details else None
    extended_date = loan_details.get('ExtendedDate', due_date).date() if 'ExtendedDate' in loan_details else due_date
    requested_amount = loan_details['requestedAmount']
    interest_amount = loan_details['intrestAmount']
    accumulated_penalty = loan_details.get('penaltyAmount', 0)
    amount_paid = loan_details.get('AmountPaid', 0)

    # Determine the final due date
    final_due_date = extended_date if extended_date else due_date
    today = datetime.now().date()

    # No penalty if today's date is before or on the final due date
    if today <= final_due_date or final_due_date is None:
        return accumulated_penalty

    # Calculate the total amount due (including any previous penalty but not including today's penalty)
    total_due = requested_amount + interest_amount + accumulated_penalty - amount_paid
    # Check if there's an outstanding amount to calculate the penalty on
    if total_due <= 0:
        return accumulated_penalty  # No additional penalty if the loan is paid up or if payment exceeds the due amount

    # Calculate the daily penalty for today
    daily_penalty = requested_amount * (penalty_percentage_per_day / 100)
    print(daily_penalty)
    # Update and return the accumulated penalty
    print(accumulated_penalty)
    updated_accumulated_penalty = accumulated_penalty + daily_penalty
    loan_details['AccumulatedPenalty'] = updated_accumulated_penalty

    return updated_accumulated_penalty


def update_penalties_in_batch(overdue_loans, product_penalty_percentages):
    batch = db.batch()
    count = 0

    for loan in overdue_loans:
        loan_details = loan.to_dict()
        product_id = loan_details.get('simpleLoanProduct')
        penalty_percentage_per_day =product_id.get().to_dict().get("PenaltyPercentageDay") # product_penalty_percentages.get(product_id, 0)


        penalty_amount = calculate_daily_penalty(loan_details, penalty_percentage_per_day)
        print(penalty_amount)

        loan_ref = db.collection('farmerSimpleLoanApply').document(loan.id)
        batch.update(loan_ref, {'penaltyAmount': penalty_amount})
        count += 1

        # Commit in batches of 500
        if count % 500 == 0:
            batch.commit()
            batch = db.batch()

    # Commit any remaining operations in the batch
    if count % 500 != 0:
        batch.commit()



# Define the Cloud Function entry point
# You can use request.args or request.json here if you need to process input data

# Call your existing functions and process the data

product_penalty_percentages = fetch_loan_products()
overdue_loans = fetch_overdue_loans()

update_penalties_in_batch(overdue_loans, product_penalty_percentages)


/usr/local/lib/python3.10/dist-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


500.0
1000.0
1500.0
1.65
0
1.65


In [ ]:
updated_loan_doc = loan_ref.get()
if updated_loan_doc.exists:
    # Access the penaltyAmount field
    print(updated_loan_doc.to_dict().get('penaltyAmount'))
else:
    print("Document does not exist.")

0


In [ ]:
overdue_loans

In [ ]:
from google.cloud import firestore
import firebase_admin
from firebase_admin import credentials, messaging, exceptions

# Initialize Firestore client
db = firestore.Client()

# Initialize Firebase Admin SDK
cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred)


In [ ]:
from firebase_admin import credentials, messaging, exceptions


In [ ]:

  user_id = "TMsWMgdVF1OvWF3xMrhGtGhsMjm2"
  user_ref = db.collection('users').document(user_id)
  # Send out a confirmation Push Notification to the Creator
  fcm_tokens = user_ref.collection('fcm_tokens').get()
  for token_doc in fcm_tokens:
    token_data = token_doc.to_dict()
    fcm_token = token_data['fcm_token']

    # Build the notification message
    message = messaging.Message(
      token=fcm_token,
      notification=messaging.Notification(
        title="request_json['title']",
        body="request_json['body']"
      ),
      android=messaging.AndroidConfig(notification=messaging.AndroidNotification(icon='firebase_default'))
    )

    # Send the notification
    try:
      response = messaging.send(message)
    except  exceptions.FirebaseError as e:
      if e.code == 'NOT_FOUND':
        # If the token is not registered, remove it from the collection
        token_doc.reference.delete()
      else:
        # Handle any other errors that may occur
        print("Error sending message:", e)
  # Log Push Notifications Being sent
  print('Push Notifications sent')

  return "Push Notifications sent", 200

Push Notifications sent


SyntaxError: ignored